# Sequential Monte Carlo - Sequential Importance Sampling

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import random
%matplotlib inline
from matplotlib import animation, rc
from IPython.display import HTML
from cycler import cycler
from sklearn.metrics import mean_squared_error

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

In [ ]:
fontsize = 40
scattersize=100
surf_labelpad = 50.

plt.style.use("seaborn")
plt.rcParams.update({'figure.figsize': (20,15), 'font.size': fontsize, 'axes.labelsize': fontsize, 'axes.labelpad': 15., 'text.usetex':True, 'xtick.labelsize': fontsize, 'xtick.major.pad': 20., 'ytick.labelsize': fontsize, 'ytick.major.pad': 20., })

colors = ['firebrick', 'navy', 'darkorange', 'olivedrab', 'cornflowerblue']


In [ ]:
rng = np.random.RandomState(26)

## Illustration of Importance Sampling

In [ ]:
def p(x):
    return 0.3*sp.stats.norm.pdf(x, loc=30, scale=10) + 0.7*sp.stats.norm.pdf(x, loc=80, scale=20)


def q(x):
    return sp.stats.norm.pdf(x, loc=50, scale=30)

In [ ]:
x = np.arange(-50, 151)
ylim = max(q(x)) 
ylim += ylim*0.2

colors = ['darkorange', 'navy', 'cornflowerblue']

fig, ax = plt.subplots()
plt.plot(x, p(x), color=colors[0], label='Target pdf, $p(z)$')
plt.legend(fontsize=fontsize, loc='upper left')
plt.xlabel('$z$')
plt.ylabel('Probability density')
ax.set_ylim(0,ylim)
plt.show()

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, p(x), color=colors[0], label='Target pdf, $p(z)$')
plt.plot(x, q(x), color=colors[1], label='Proposal pdf, $q(z)$')
plt.xlabel('$z$')
plt.ylabel('Probability density')
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_ylim(0,ylim)
plt.show()

In [ ]:
def plot_proposal(fig, ax, x, pdf_target, pdf_proposal, z, q_z, p_z, accepted=None, i=None, colors=None, plot_pz=True):   
    if colors is None:
        colors = ['darkorange', 'navy', 'cornflowerblue']
    ylim = max(pdf_proposal)
    ylim += ylim*0.05

    ax.cla()

    if i is not None:
        fig.suptitle('Iteration %i' % (i + 1))
    if accepted is not None:
        color = 'g' if accepted else 'r'
    else:
        color = 'k'
        
    # Plot prior
    ax.plot(x, pdf_target, color=colors[0], linewidth=2.5, label='Target pdf, $p(z)$')
    ax.plot(x, pdf_proposal, color=colors[1], linewidth=2.5, label='Proposal pdf, $q(z)$')

    # Stem plot of current sample and proposal:
    ax.plot([z]*2, [0, q_z], color=color)
    ax.scatter(z, q_z, s=100, color=color)
    
    if plot_pz:
        ax.plot([z]*2, [0, p_z], color=color, linestyle='dashed')
        ax.scatter(z, p_z, s=200, color=color)

    ax.set(xlabel='$z$', ylabel='Probability Density')
    ax.set_ylim(0,ylim)
    plt.legend(fontsize=fontsize, loc='upper left')

    return ax

In [ ]:
z = 75

fig, ax = plt.subplots()
plot_proposal(fig, ax, x, p(x), q(x), z, q(z), p(z), plot_pz=False)
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_ylim(0,ylim)
ax.text(z+2, q(z)*.95, '$q(\\tilde{z}^{(1)}) = '+str(round(q(z),3))+'$')

In [ ]:
fig, ax = plt.subplots()
plot_proposal(fig, ax, x, p(x), q(x), z, q(z), p(z), plot_pz=True)
ax.text(z+2, p(z)*1.05, '$p(\\tilde{z}^{(1)}) = '+str(round(p(z),3))+'$')
ax.text(z+2, q(z)*.95, '$q(\\tilde{z}^{(1)}) = '+str(round(q(z),3))+'$')
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_ylim(0,ylim)

print(p(z))
print(np.random.uniform(0, q(z)))

In [ ]:
import pylab as px

def half_brace(x, beta=10.):
    x0, x1 = x[0], x[-1]
    y = 1/(1.+np.exp(-1*beta*(x-x0))) + 1/(1.+np.exp(-1*beta*(x-x1)))
    return y


In [ ]:
fig, ax = plt.subplots()
plot_proposal(fig, ax, x, p(x), q(x), z, q(z), p(z), plot_pz=True)
ax.text(z+2, p(z)*1.05, '$p(\\tilde{z}^{(1)}) = '+str(round(p(z),3))+'$')
ax.text(z+2, q(z)*.95, '$q(\\tilde{z}^{(1)}) = '+str(round(q(z),3))+'$')
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_ylim(0,ylim)

xstep = 100
ybrace = q(z) + np.linspace(0, (p(z)-q(z))/2, xstep//2)
x0 = half_brace(ybrace, beta=1e5)
xbrace = np.concatenate((x0, x0[::-1]))
plt.plot(z+2 + xbrace, np.linspace(q(z), p(z), xstep), linewidth=2.5, color='k')
ax.text(z+6, q(z) + (p(z)-q(z))/2, '${p(\\tilde{z}^{(1)})}/{q(\\tilde{z}^{(1)})} = '+str(round(p(z)/q(z),3))+'$')


In [ ]:
fig, ax = plt.subplots()
plot_proposal(fig, ax, x, p(x), q(x), z, q(z), p(z), plot_pz=True)
ax.text(z+2, p(z)*1.05, '$p(\\tilde{z}^{(1)})$')
ax.text(z+2, q(z)*.95, '$q(\\tilde{z}^{(1)})$')
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_ylim(0,ylim)

xstep = 100
ybrace = q(z) + np.linspace(0, (p(z)-q(z))/2, xstep//2)
x0 = half_brace(ybrace, beta=1e5)
xbrace = np.concatenate((x0, x0[::-1]))
plt.plot(z+2 + xbrace, np.linspace(q(z), p(z), xstep), linewidth=2.5, color='k')
ax.text(z+6, q(z) + (p(z)-q(z))/2, '${p(\\tilde{z}^{(1)})}/{q(\\tilde{z}^{(1)})} = '+str(round(p(z)/q(z),3))+'$')

# Plot second sample:
z1 = 50
plt.plot([z1]*2, [0, q(z1)], color='red')
plt.scatter(z1, q(z1), s=200, color='red')
ax.text(z1-5, p(z1)*.87, '$p(\\tilde{z}^{(2)})$')
ax.text(z1-5, q(z1)*1.03, '$q(\\tilde{z}^{(2)})$')
plt.plot([z1]*2, [0, p(z1)], linestyle='dashed', color='red')
plt.scatter(z1, p(z1), s=200, color='red')


In [ ]:
fig, ax = plt.subplots()
plot_proposal(fig, ax, x, p(x), q(x), z, q(z), p(z), plot_pz=True)
ax.text(z+2, p(z)*1.05, '$p(\\tilde{z}^{(1)})$')
ax.text(z+2, q(z)*.95, '$q(\\tilde{z}^{(1)})$')
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_ylim(0,ylim)

xstep = 100
ybrace = q(z) + np.linspace(0, (p(z)-q(z))/2, xstep//2)
x0 = half_brace(ybrace, beta=1e5)
xbrace = np.concatenate((x0, x0[::-1]))
plt.plot(z+2 + xbrace, np.linspace(q(z), p(z), xstep), linewidth=2.5, color='k')
ax.text(z+6, q(z) + (p(z)-q(z))/2, '${p(\\tilde{z}^{(1)})}/{q(\\tilde{z}^{(1)})} = '+str(round(p(z)/q(z),3))+'$')

# Plot second sample:
z1 = 50
plt.plot([z1]*2, [0, q(z1)], color='red')
plt.scatter(z1, q(z1), s=200, color='red')
ax.text(z1-5, p(z1)*.87, '$p(\\tilde{z}^{(2)})$')
ax.text(z1-5, q(z1)*1.03, '$q(\\tilde{z}^{(2)})$')
plt.plot([z1]*2, [0, p(z1)], linestyle='dashed', color='red')
plt.scatter(z1, p(z1), s=200, color='red')

xstep = 100
ybrace = q(z1) + np.linspace(0, (p(z1)-q(z1))/2, xstep//2)
x0 = half_brace(ybrace, beta=1e5)
xbrace = np.concatenate((x0, x0[::-1]))
plt.plot(z1-6 + xbrace, np.linspace(q(z1), p(z1), xstep), linewidth=2.5, color='r')
ax.text(z1-80, q(z1) + (p(z1)-q(z1))/2, '${p(\\tilde{z}^{(2)})}/{q(\\tilde{z}^{(2)})} = '+str(round(p(z1)/q(z1),3))+'$')

## Example 1: Non-Markovian

In [ ]:
def prior(x, x_prev, phi, q):
    return sp.stats.norm.pdf(x, phi*x_prev, np.sqrt(q))

def likelihood(y, x_traj, beta, r):
    mean = 0
    for k in range(1,t+1):
        print('t: {}, k: {}'.format(t,k))
        mean += beta**(t-k)*x_traj[k]
    return sp.stats.norm.pdf(y, mean, np.sqrt(r)), mean

Generate data from non-Markovian model:

In [ ]:
colors = ['firebrick', 'navy', 'darkorange', 'olivedrab', 'cornflowerblue']


n_particles = 200
T = 100
phi = 0.9
q = 1
beta = 0.5
r = 1

x = np.zeros ((T,))
y = np.zeros((T,))
x[0] = np.sqrt(q)*np.random.randn()
for t in range(T):
    x[t] = phi * x[t-1]+ np.sqrt(q) * np.random.randn()
    for k in range(1,t+1):
        y[t] += (beta**(t-k))*x[k]
    y[t] += np.sqrt(r) * np.random.randn()

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, label='$z_t$', color=colors[1])
plt.plot(y, label='$x_t$', color=colors[0], marker='o')
plt.xlabel('Time index, $t$')
plt.ylabel('Signal amplitude')
plt.legend(fontsize=fontsize, loc='upper left')
ax.set_xlim(0,T)
ylim = ax.get_ylim()

Sample from prior:

$w(\mathbf{z}_{1:t}) = w(\mathbf{z}_{1:t-1})\, p(y_t\,|\,\mathbf{x}_{1:t})$

In [ ]:
def SIS(y, q, r, beta, phi, T, N):
    logw = 1/N*np.ones((T,N))
    mu = 0
    samples = np.zeros((T,N))
    lik_mean = np.zeros((N,))
    for t in range(T):
        # Insert code here. Hint:
        # samples[t,:] = ?
        # lik_mean = ?
        # logw[t,:] = ?
        
    return samples, np.exp(logw)

In [ ]:
SIS_samples, SIS_weights = SIS(y, q, r, beta, phi, T, 1000000)

### Point Estimation

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, label='$z_t$', color=colors[1])
plt.plot(y, label='$x_t$', color=colors[0], marker='o')

# MAP estimates:
MAP_SIS = np.zeros((T,))
for t in range(T):
    MAP_SIS[t] = SIS_weights[t,:].transpose() @ SIS_samples[t,:]
plt.plot(MAP_SIS, label='$\hat{z}_t^{SIS}$', color=colors[2], linewidth=2.5, linestyle='dashed')

plt.xlabel('Time index, $t$')
plt.ylabel('Signal amplitude')
ax.set_xlim(0,T)
plt.legend(fontsize=fontsize, loc='upper left')

### Weight degeneracy

In [ ]:
SIS_samples, SIS_weights = SIS(y, q, r, beta, phi, T, n_particles)

fig, ax = plt.subplots()
colorwheel = [plt.cm.viridis(i) for i in np.linspace(0, 1, n_particles)]
ax.set_prop_cycle(cycler('color', colorwheel))
h = []
for n in range(n_particles):
    h = plt.scatter(np.linspace(0,T, T), [n]*T, s=SIS_weights[:,n]*1000)
plt.xlabel('Time index, $t$')
plt.ylabel('Particle index, $\ell$')
ax.set_xlim(0,T)
ax.set_ylim(0,n_particles)
ax.set_xlim(-1,T)
ax.set_ylim(-1,n_particles)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
ax.set_xticks(xticks[1:-1])
ax.set_yticks(yticks[1:-1])

### Effective Sample Size

In [ ]:
# Compute effective sample size here:
# ESS_SIS = ?

fig, ax = plt.subplots()
plt.plot(np.linspace(0,T-1,T), ESS_SIS, color=colors[0], marker='o', label='ESS')
plt.plot([0, T-1], [n_particles]*2, label='Number of particles, $L$', color='k', linestyle='dashed')
plt.xlabel('Time index, $t$')
plt.ylabel('Number of particles, $\ell$')
ax.set_xlim(0,T)
ax.set_ylim(0, max(np.max(ESS_SIS), n_particles)*1.05)
plt.legend(fontsize=fontsize)

## Sequential Monte Carlo

In [ ]:
def systematic_resampling(w):
    # Insert code for systematic resampling here. 
    return ind

In [ ]:
w = np.random.rand(10,)
ind = systematic_resampling(w)

In [ ]:
def SIR(y, q, r, beta, phi, T, N, ):
    logw = 1/N*np.ones((T,N))
    logw_raw = 1/N*np.ones((T,N))
    mu = 0
    samples = np.zeros((T,N))
    resampled_path = np.zeros((T,N))
    lik_mean = np.zeros((N,))
    for t in range(T):        
        # Insert code here. Hint:
        # samples[t,:] = ?
        # lik_mean = ?
        # logw[t,:] = ?
        # resampled_path[t,:]
                
    return samples, np.exp(logw_raw), np.exp(logw), resampled_path

In [ ]:
SIR_samples, _, SIR_weights, resampled_path = SIR(y, q, r, beta, phi, T, 10000)

### Point estimation

In [ ]:
plot_include = ['z', 'x', 'SIS', 'SIR']

# SIR MAP estimate:
MAP_SIR = np.zeros((T,))
var_SIR = np.zeros((T,))
for t in range(T):
    MAP_SIR[t] = SIR_weights[t,:].transpose() @ SIR_samples[t,:]

ylim = [np.min([MAP_SIR,MAP_SIS,x,y])*1.05, np.max([MAP_SIR,MAP_SIS,x,y])*1.05]
xlim = [0, T-1]    

for includes in range(1,len(plot_include)+1):
    fig, ax = plt.subplots()
    
    if 'z' in plot_include[:includes]:
        plt.plot(x, label='$z_t$', color=colors[1])
    if 'x' in plot_include[:includes]:
        plt.plot(y, label='$x_t$', color=colors[0], marker='o')
        # plt.scatter(np.linspace(0,T-1,T), y, label='$x_t$')
    if 'SIR' in plot_include[:includes]:
        plt.plot(MAP_SIR, label='$\hat{z}_t^{SIR}$', linewidth=2.5, color=colors[3])
    if 'SIS' in plot_include[:includes]:
        plt.plot(MAP_SIS, label='$\hat{z}_t^{SIS}$', linewidth=2.5, linestyle='dashed', color=colors[2])

    plt.legend(fontsize=fontsize, loc='lower left')
    plt.xlabel('Time index, $t$')
    plt.ylabel('Signal amplitude')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)


In [ ]:
plot_include = ['SIS', 'SIR']

for includes in range(1,len(plot_include)+1):
    fig, ax = plt.subplots()
    if 'SIR' in plot_include[:includes]:
        plt.plot(np.abs(x-MAP_SIR)**2, color=colors[3], linewidth=2.5, label='SIR')
    if 'SIS' in plot_include[:includes]:
        plt.plot(np.abs(x-MAP_SIS)**2, color=colors[2], linestyle='dashed', linewidth=2.5, label='SIS')
    plt.xlabel('Time index, $t$')
    plt.ylabel('Error $|z_t - \hat{z}_t|^2$')
    plt.legend(fontsize=fontsize)
    ax.set_xlim(0, T-1)

In [ ]:
SIR_samples, _, SIR_weights, resampled_path = SIR(y, q, r, beta, phi, T, n_particles)

fig, ax = plt.subplots()
colorwheel = [plt.cm.viridis(i) for i in np.linspace(0, 1, n_particles)]
ax.set_prop_cycle(cycler('color', colorwheel))
h = []
for n in range(n_particles):
    h = plt.scatter(np.linspace(0,T, T), [n]*T, s=SIR_weights[:,n]*1000)
plt.xlabel('Time index, $t$')
plt.ylabel('Particle index, $\ell$')
ax.set_xlim(0,T)
ax.set_ylim(0,n_particles)
ax.set_xlim(-1,T)
ax.set_ylim(-1,n_particles)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
ax.set_xticks(xticks[1:-1])
ax.set_yticks(yticks[1:-1])

### Effective Sample Size (ESS)

In [ ]:
# Evaluate ESS here:
# ESS_SIR = ?

plot_includes = ['L', 'SIS', 'SIR']

for includes in range(1,len(plot_includes)+1):
    fig, ax = plt.subplots()
    if 'L' in plot_includes[:includes]:
        plt.plot([0, T-1], [n_particles]*2, label='Number of particles, $L$', color='k', linestyle='dashed')
    if 'SIS' in plot_includes[:includes]:
        plt.plot(np.linspace(0,T-1,T), ESS_SIS, color=colors[0], marker='o', label='ESS SIS')
    if 'SIR' in plot_includes[:includes]:
        plt.plot(np.linspace(0,T-1,T), ESS_SIR, color=colors[1], marker='*', markersize=10, label='ESS SIR')
    plt.xlabel('Time index, $t$')
    plt.ylabel('Number of particles, $\ell$')
    ax.set_xlim(0,T)
    ax.set_ylim(0, max(np.max(ESS_SIR), n_particles)*1.05)
    plt.legend(fontsize=fontsize, loc = 'lower right')
    

### Path Degeneracy of SIR

In [ ]:
T_path = 20
n_particles_path = 5
SIR_samples, _, SIR_weights, resampled_path = SIR(y, q, r, beta, phi, T_path, n_particles_path)


fig, ax = plt.subplots()
colorwheel = [plt.cm.viridis(i) for i in np.linspace(0, 1, n_particles_path)]
ax.set_prop_cycle(cycler('color', colorwheel))
h = []
for n in range(n_particles_path):
    h = plt.scatter(np.linspace(0,T_path-1, T_path), [n]*T_path, s=SIR_weights[:,n]*1000)
    for t in range(0,T_path-1):
        plt.plot([t,t+1], [resampled_path[t,n],n], 'k')
plt.xlabel('Time index, $t$')
plt.ylabel('Particle index, $\ell$')
ax.set_xlim(-1,T_path)
ax.set_ylim(-1,n_particles_path)
xticks = ax.get_xticks()
yticks = ax.get_yticks()
ax.set_xticks(xticks[1:-1])
ax.set_yticks(yticks[1:-1])